# Scop3P

A comprehensive database of human phosphosites within their full context. Scop3P integrates sequences (UniProtKB/Swiss-Prot), structures (PDB), and uniformly reprocessed phosphoproteomics data (PRIDE) to annotate all known human phosphosites. 

Scop3P, available at https://iomics.ugent.be/scop3p, presents a unique resource for visualization and analysis of phosphosites and for understanding of phosphosite structure–function relationships.

## Dependencies

In [ ]:
%%capture
!pip install pandas matplotlib py3Dmol

In [ ]:
import requests
import pandas as pd 
import py3Dmol

## Target protein

In [ ]:
TARGET_PROTEIN_ID = "O00571" # Write here the Protein ID of your protein of interest
PDB_ID = "4PXA" # Write here the PDB ID of your protein of interest

## API Request

This function makes a GET request to SCOP3P API endpoint for a given protein accession ID and returns the protein sequence in string format.

In [ ]:
def fetch_protein_modifications(accession):
    """
    Fetches protein modifications for a given UniProt ID.

    Parameters:
    accession (str): UniProt ID of the protein.

    Returns:
    dict: A dictionary containing protein modifications.
    """
    BASE_URL = "https://iomics.ugent.be/scop3p/api/modifications"
    url = f'{BASE_URL}?accession={accession}'
    headers = {'accept': 'application/json'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None

## Data parsing

For parsing the JSON as a table, we'll use pandas library.

In [ ]:
scop3P_results = fetch_protein_modifications(TARGET_PROTEIN_ID)

protein_name = scop3P_results['proteinName']
entry_name = scop3P_results['entryName']
accession = scop3P_results['accession']
url = scop3P_results['url']
modifications = scop3P_results['modifications']

print("""
--------------------------------------------------------------------------------
Scop3P: A Comprehensive Resource of Human Phosphosites within Their Full Context
--------------------------------------------------------------------------------

{0}:{1}

Phospho-sites found: {3} entries.

Full entry available on SCOP3P website: {2}
""".format(entry_name, protein_name, url, len(modifications)))

In [ ]:
def get_modifications_table(modifications):
    """
    Displays the protein modifications in a pandas DataFrame.

    Parameters:
    modifications (list): A list of dictionaries, each representing a protein modification.
    """
    df = pd.DataFrame(modifications)
    df = df[['residue', 'name', 'evidence', 'position', 'source', 'reference', 'functionalScore', 'specificSinglyPhosphorylated']]
    
    return df 
    
modifications_table = get_modifications_table(modifications)
display(modifications_table)

## Rendering results

For visualizing the 3D structure, we'll use py3Dmol library.

In [ ]:
def display_3D(modifications_table):
    view = py3Dmol.view(query=f'pdb:{PDB_ID}')
    
    view.setStyle({'cartoon': { 'color': 'skyblue' }})
    view.addSurface(py3Dmol.VDW, {'opacity': 0.60, 'color': 'white' })
    
    for index, row in modifications_table.iterrows():
        position = row['position']
        residue = row['residue']
        
        if residue == 'PhosphoY':
            color = '#2CA02C'
        elif residue == 'PhosphoS':
            color = '#1F77B4'
        elif residue == 'PhosphoT':
            color = '#FF7F0E'
        else:
            color = '#7B241C'
        
        view.addSurface(py3Dmol.VDW, {'opacity': 1.0, 'color': color }, {'resi': [str(position)]})
        view.setHoverable(
            {},
            True,
            """
            function(atom, viewer, event, container) {{
                if (!atom.label) {{
                    atom.label = viewer.addLabel(
                        "{0} " + atom.resn + {1}, {{ 'position': atom, backgroundColor: 'mintcream', fontColor:'black' }}
                    );
                }}
            }}
            """.format(residue, position),
            """
            function(atom,viewer) {
                if(atom.label) {
                    viewer.removeLabel(atom.label);
                    delete atom.label;
                }
            }
            """
        )
    
    view.zoomTo()
    return view

In [ ]:
display_3D(modifications_table)